In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F
import torchvision.transforms as transforms

import numpy as np

In [2]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        # Inspired by the implementation of binary_cross_entropy_with_logits
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

        # This formula gives us the log sigmoid of 1-p if y is 0 and of p if y is 1
        invprobs = F.logsigmoid(-input * (target * 2 - 1))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.mean()

In [3]:
# SorensonDiceCoefficient
class SorensonDiceLoss(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, logits, labels, smoothing=1):
                
        batches = logits.size(0)
        flat_logits = logits.view((batches, -1))
        flat_labels = labels.view((batches, -1))
        
        intersection = 2. * (flat_logits * flat_labels).sum(1) + smoothing
        union = flat_logits.sum(1) + flat_labels.sum(1) + smoothing
        
        return 1 - intersection / union

In [4]:
class BinaryCrossEntropy(nn.Module):
    
    def __init__(self, weight=None, size_average=True):
        super().__init__()
        self.bce_loss = nn.BCELoss(weight, size_average)

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        probs_flat = probs.view(-1)  # Flatten
        targets_flat = targets.view(-1)  # Flatten
        return self.bce_loss(probs_flat, targets_flat)

In [35]:
sample_logits = torch.tensor([0.1, 0.2, 0.4, 0.3, 0.1]).unsqueeze(0)
sample_targets = torch.tensor([0., 1., 0.0, 0., 1.]).unsqueeze(0)

new_acc = BinaryCrossEntropy().forward(sample_logits, sample_targets)
print(new_acc)

new_acc = SorensonDiceLoss().forward(sample_logits, sample_targets)
print(new_acc)

new_acc = FocalLoss(0.2).forward(sample_logits, sample_targets)
print(new_acc)

tensor(0.7509)
tensor([0.6098])
tensor(0.6616)


In [33]:
# Pytorch Implementation
sample_logits = torch.tensor([0.1, 0.2, 0.4, 0.3, 0.1]).unsqueeze(0)
sample_targets = torch.tensor([0, 0, 0, 0, 1]).unsqueeze(0)

py_acc = F.cross_entropy(sample_logits, sample_targets)

NameError: name 'Variable' is not defined

In [34]:
criterion = nn.CrossEntropyLoss()

output = torch.randn(1, 10).float()
target = torch.FloatTensor(1).uniform_(0, 2).long()

print(output, target)

loss = criterion(output, target)
print(loss)

tensor([[ 0.5802, -0.0760,  0.7653, -1.0983,  0.9196, -1.6029,  0.1051,  0.2443,
          0.4650,  0.7618]]) tensor([0])
tensor(2.0608)
